In [1]:
# 1.批量转换json标注文件to mask
import os
path = 'E:\\sc\\image_data\\meter_img\\json' # path为json文件存放的路径 
json_file = os.listdir(path) 
os.system("activate labelme") 
for file in json_file: 
    os.system("labelme_json_to_dataset.exe %s"%(path + '/' + file))

In [12]:
# 2.批量修改img和mask文件名并提取出来
import os
import glob
import shutil

if __name__ == "__main__":
    path = 'E:\\sc\\image_data\\meter_img\\json'
    imgs_paths = glob.glob('E:\\sc\\image_data\\meter_img\\json\\*_json')
    # print(imgs_paths)
    save_img_path = 'E:\\sc\\image_data\\meter_img\\train_img'
    save_mask_path = 'E:\\sc\\image_data\\meter_img\\mask'
    for image_path in imgs_paths:
        image_name = image_path.split('\\')[-1]
        oldname_img = image_path + '\\' +'img.png'
        newname_img = save_img_path + '\\' + image_name + '.jpg'
        shutil.copyfile(oldname_img,newname_img)
        oldname_mask = image_path + '\\' +'label.png'
        newname_mask = save_mask_path + '\\' + image_name + '.png'
        shutil.copyfile(oldname_mask,newname_mask)
        # print(image_path)
        # print(image_name)


In [18]:
# 3.批量修改mask
import cv2
import os
import numpy as np
import glob
import random
# from PIL import Image

if __name__ == "__main__":
    imgs_paths = glob.glob('E:\\sc\\image_data\\meter_img\\mask/*.png')
    for image_path in imgs_paths:
        savepath = image_path.replace('mask', 'train_mask')
        label = cv2.imread(image_path)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        print(label.shape)

        # 膨胀刻度
        dilation_img = label.copy()
        indax_point = np.where(label == 38)
        dilation_img[indax_point] = 0
        indax_scale = np.where(label == 75)
        dilation_img[indax_scale] = 255
        kernelsize = 3  # magic number
        kernel = np.ones((kernelsize,kernelsize), np.uint8)
        dilation_img = cv2.dilate(dilation_img, kernel, iterations=1)
        # visaulize
        # cv2.imwrite(image_path.replace('json', 'json_dilation'), dilation_img)

        # 转成训练标准mask
        indax_point = np.where(label == 38)
        label[indax_point] = 1
        # indax_scale = np.where(label == 75)
        indax_scale = np.where(dilation_img == 255)
        label[indax_scale] = 2
        cv2.imwrite(savepath, label)
    print('end')


(509, 502)
(482, 497)
(287, 279)
(529, 552)
(418, 372)
(477, 476)
(330, 331)
(281, 283)
(327, 323)
(384, 398)
(288, 284)
(328, 341)
(756, 777)
(279, 287)
(414, 430)
(347, 345)
(431, 421)
(504, 510)
(471, 466)
(737, 743)
(329, 342)
(541, 545)
(387, 390)
(409, 416)
(336, 330)
(287, 280)
(342, 343)
(323, 324)
(421, 419)
(336, 334)
(327, 338)
(349, 353)
(441, 446)
(343, 342)
(343, 342)
end


In [31]:
# 4.复制到voc数据集中，批量添加图像文件名到voc数据集的trainval.txt, train.txt 和val.txt中
import os
import glob
import shutil
import random

if __name__ == '__main__':
    imgs_paths = glob.glob('E:\\sc\\image_data\\meter_img\\train_img\\*.jpg')
    to_img_path = 'E:\\sc\\image_data\\meter\\meter_seg_voc\\JPEGImages\\'
    masks_paths = glob.glob('E:\\sc\\image_data\\meter_img\\train_mask\\*.png')
    to_mask_path = 'E:\\sc\\image_data\\meter\\meter_seg_voc\\SegmentationClass\\'
    txt_path = 'E:\\sc\\image_data\\meter\\meter_seg_voc\\ImageSets\\Segmentation'
    imgname_list = []

    # 复制
    for image_path in imgs_paths:
        oldname_img = image_path
        newname_img = to_img_path + image_path.split('\\')[-1]
        shutil.copyfile(oldname_img,newname_img)
        imgname_list.append(image_path.split('\\')[-1].split('.')[0])
    for mask_path in masks_paths:
        oldname_mask = mask_path
        newname_mask = to_mask_path + mask_path.split('\\')[-1]
        shutil.copyfile(oldname_mask,newname_mask)
    # print(imgname_list)

    # 添加txt
    file_trainval = open(txt_path + '\\trainval.txt', 'a')
    for name in imgname_list:
        file_trainval.writelines(name)
        file_trainval.writelines('\n')
    file_trainval.close()

    file_train = open(txt_path + '\\train.txt', 'a')
    file_val = open(txt_path + '\\val.txt', 'a')
    proportion = 0.3  # val 占比
    for name in imgname_list:
        if random.random() > proportion:
            file_train.writelines(name)
            file_train.writelines('\n')
        else:
            file_val.writelines(name)
            file_val.writelines('\n')
    file_train.close()
    file_val.close()

In [55]:
# 查看voc数据集mask
import os
import glob
import cv2


if __name__ == '__main__':
    imgs_paths = glob.glob('E:\\sc\\image_data\\meter\\meter_seg_voc\\SegmentationClass\\*.png')
    mask_visaulize_path = 'E:\\sc\\image_data\\meter\\meter_seg_voc\\mask_v\\'
    # print(len(imgs_paths))
    for image_path in imgs_paths:
        mask_name = image_path.split('\\')[-1]
        img_mask = cv2.imread(image_path,0)
        img = np.zeros((img_mask.shape[0],img_mask.shape[1],3))
        
        indax_point = np.where(img_mask == 1)
        img[indax_point] = (0,0,255)
        indax_scale = np.where(img_mask == 2)
        img[indax_scale] = (255,0,0)
        cv2.imwrite(mask_visaulize_path + mask_name, img)

In [ ]:
# 批量修改原数据集中train.txt, val.txt中图像路径 imgpath_repair.py
import os

def changepath(path,topath):
    file = open(path, 'r')
    lines = []
    for i in file:
        lines.append(i)  #逐行将文本存入列表lines中
    file.close()

    new = []
    for line in lines:  # 逐行遍历
        line = line.replace('images/train/','')
        line = line.replace('images/val/','')
        p = 0  # 定义计数指针
        for bit in line:  # 对每行进行逐个字遍历
            if bit == '.':   # 遇到0时进行处理（我们可以修改成我们想要的定位）
                line = line[:p]
                break
            else:
                p = p + 1  # 如果bit不是空格，指针加一
        new.append(line)
    
    file_write = open(topath, 'w')
    for var in new:
        file_write.writelines(var)
        file_write.writelines('\n')
    file_write.close()


if __name__ == '__main__':
    changepath('E:/sc/image_data/meter/meter_seg_voc/ImageSet/Segmentation/val_yuan.txt'
               , 'E:/sc/image_data/meter/meter_seg_voc/ImageSet/Segmentation/val.txt')


In [ ]:
# 批量改文件名
import os
# path=input('请输入文件路径(结尾加上/)：')
# path = 'imgs/'

#获取该目录下所有文件，存入列表中
fileList=os.listdir(path)

n=0
for i in fileList:
    
    #设置旧文件名（就是路径+文件名）
    oldname=path+ os.sep + fileList[n]   # os.sep添加系统分隔符
    
    #设置新文件名
    newname=path + os.sep +'ture'+str(n+1)+'.jpg'
    
    os.rename(oldname,newname)   #用os模块中的rename方法对文件改名
    print(oldname,'======>',newname)
    
    n+=1